## 作業工程計画

1. 受領データのシートを一つずつ読み込んで、一つのデータフレームに入れる
2.　ヘッダーをシートごとに調整する
3.　複数のシートを一つに結合する


### 参考サイト
XXX

## ライブラリ

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import re 

## 村上さん pandas_tool

In [2]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

# データ1の前処理

## エクセルデータの読み込み

### データ読み込み

In [3]:
"""
複数のシートを持つエクセルのパスを渡すと、シート名を列名として追加して一つに結合したデータフレームを返す
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/①設問情報（分野、文字数等の情報、企画ペース）_0522更新v1.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
# sn_list = sheet_names_list[0:len(sheet_names_list)-2]
#配列ループ
df = [] #空のリスト
for i in (0,1): #21冬

    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i], header=0, index_col=None,skiprows=[1])#headerを修正
    ret = ret.rename(columns={'企画ペース\n完成版': '企画ペース',
                             '文字数\nＳ枠数': '文字数',
                             'S枠（数字）': '割当ごとのS枠数',
                             '割当単位':'割当'})#冬だけ
    ret["年度"]= sn_list[i][0:2]#sn_listのiつ目の要素から3番目の文字
    ret["採点回"]= sn_list[i][2]#sn_listのiつ目の要素から3番目の文字
    ret["学年"]= sn_list[i][3]#sn_listのiつ目の要素から4番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)
    
for i in (2,3,4): #21秋
    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i], header=4,skiprows=[5], index_col=None)#headerを修正　,skiprows=[0,1,2,3,5]
    ret["年度"]= sn_list[i][0:2]#sn_listのiつ目の要素から3番目の文字
    ret["採点回"]= sn_list[i][2]#sn_listのiつ目の要素から3番目の文字
    ret["学年"]= sn_list[i][3]#sn_listのiつ目の要素から4番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)    

for i in (5,6): #21夏1〜2
    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i],header = 1, index_col=None)#headerを修正
    ret["年度"]= sn_list[i][0:2]#sn_listのiつ目の要素から3番目の文字
    ret["採点回"]= sn_list[i][2]#sn_listのiつ目の要素から3番目の文字
    ret["学年"]= sn_list[i][3]#sn_listのiつ目の要素から4番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)
    
for i in range(7,len(sn_list)): #21夏3〜
    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i],header = 1, index_col=None)#headerを修正
    ret["年度"]= sn_list[i][0:2]#sn_listのiつ目の要素から3番目の文字
    ret["採点回"]= sn_list[i][2]#sn_listのiつ目の要素から3番目の文字
    ret["学年"]= sn_list[i][3]#sn_listのiつ目の要素から4番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)

"""
集計したデータを縦に結合する。
"""
dfs = df[0] #集計結果を一つのDFに結合
for i in df[1:]:
    dfs = dfs.append(i)

# rename    
dfs = dfs.rename(columns={'科目分類名': '科目'})
# dfs = dfs.rename(columns={'Unnamed: 6': '行の文字数'})

# dfs = dfs[['科目','科目コード', '採点回', '学年', '割当','分野', '企画ペース', '最終ペース']]
dfs = dfs.rename(columns={'割当':'割当1'})

#データ型の変更
dfs['割当1'] = dfs.loc[:, "割当1"].astype('str')
dfs['年度'] = dfs.loc[:, "年度"].astype('str')


#置換
map_dictionary ={"国語" : 10, "数学" :20, "英語" : 30} 
dfs['科目コード'] = dfs['科目'].map(map_dictionary) 

dfs = dfs.replace({'最終ペース': {'ー':np.nan}})
dfs['最終ペース'] = dfs.loc[:, "最終ペース"].astype('float64')

#空白行の削除
dfs= dfs[~dfs["科目コード"].isna()]

#割当から客観とnanを削除
df = []
for i in dfs['割当1']:
    if 'K' in i:
        i = np.nan
    df.append(i)
dfs['割当1'] = df
dfs =dfs[dfs['割当1'] != "nan"]
dfs= dfs[~dfs["割当1"].isna()]

raw_df = dfs

In [82]:
df_1[(df_1['採点回']=="夏")&(df_1['学年']=="2")&(df_1['年度']=='21')].head(5)

,科目,大問番号,設問番号,割当1,分野,文字数,行の文字数,割当ごとのS枠数,ポイント数,配点,企画ペース,最終ペース,年度,採点回,学年,S枠\n(数字),科目コード
0,国語,1,3,01,評論,60,NaN,1.0,3,5,300.0,181.7,21,夏,2,NaN,10.0
1,国語,1,5,02,ポ評論,70,NaN,1.0,3,8,300.0,136.4,21,夏,2,NaN,10.0
2,国語,2,4,03,ポ小説,60,NaN,1.0,3,5,270.0,159.9,21,夏,2,NaN,10.0
3,国語,3,3,04,古文,70,NaN,1.0,3,8,290.0,121.4,21,夏,2,NaN,10.0
4,国語,3,4,05,古文,一行,17.0,1.0,3,5,550.0,427.9,21,夏,2,NaN,10.0


## データクレンジング

In [60]:
# df_1.columns.tolist()

In [83]:
# df = df_1
# df['文字数'].unique()

array(['60文字', '40文字', '1行', '2行', '1行（書き下し文）', '40文字（説明）', '50文字', nan,
       '※解答枠６。まとめて10点', '10文字＋20文字', '15文字程度', '5語程度×2枠', '30字以上40字以内',
       '30語程度', '20文字', '70文字', '１行', '10文字', 2, 1, 3, '10語', '10語程度',
       '30字程度（日本語）', '50字以上60字以内（日本語）', '70字程度（日本語）', '２行', '45語程度', 70,
       30, 80, 40, 20, 50, '1行(35字）', '５語', '２行(２０語程度）', '30語', '5語',
       '30字', '10字', '2行（11－13字）', '1行（5－6字）', '30～45語', 60, 75, 120, 100,
       25, '３行', '４行', 35, '3行', '一行', 15, '25+25', '2行＋2行', 90, '4行', 18,
       '\u3000', '単語？', '10+20', '15+20', '3行？', 45, '2枠', '1枠', '3枠',
       '15+20字', '6語', '35字', '20語', '5～10語', '15字', '12語', '45語', '太枠',
       'ポ採\u30001枠', 'ポ採\u30003枠', '15語', '小枠2つ', 12, '1~2行', 85],
      dtype=object)

In [61]:
# df_u = df_1
# df_u['文字数'].unique()
# x = df_u['文字数'].unique()
# y = []
# z = []
# for i in x:
#     if '行' not in str(i):
#       y.append(i)
# for i in y:
#     if isinstance(i, int)== False:
#         z.append(i)
# print(z)

['60文字', '40文字', '40文字（説明）', '50文字', nan, '※解答枠６。まとめて10点', '10文字＋20文字', '15文字程度', '5語程度×2枠', '30字以上40字以内', '30語程度', '20文字', '70文字', '10文字', '10語', '10語程度', '30字程度（日本語）', '50字以上60字以内（日本語）', '70字程度（日本語）', '45語程度', '５語', '30語', '5語', '30字', '10字', '30～45語', '25+25', '\u3000', '単語？', '10+20', '15+20', '2枠', '1枠', '3枠', '15+20字', '6語', '35字', '20語', '5～10語', '15字', '12語', '45語', '太枠', 'ポ採\u30001枠', 'ポ採\u30003枠', '15語', '小枠2つ']


In [5]:
"""
文字数


"""

def make_sum(input_str):
    if input_str =="":
        return np.nan
    else:
        input_list = input_str.split('＋')#+が半角か全角か異なる場合もあり。
        input_list_num = [int(s) for s in input_list] #リスト内包表記　復習
        return sum(input_list_num)
#5倍するラムダ関数
times_five = lambda x: x*5

#一行に対するルールを定義する 上から順に作成して、順次テストする
x = ['文字程度','文字（説明）','字程度（日本語）','文字','※解答枠６。まとめて10点', '単語？','太枠', 'ポ採\u30001枠', 'ポ採\u30003枠','\u3000','小枠2つ','2枠', '1枠', '3枠']
y = ["語程度","語"]
z = ['5語程度×2枠', '50字以上60字以内（日本語）','30字以上40字以内']
def make_mojisu(df):
    
    #行を含む場合は行数の文字列を代入
    if '行' in str(df['文字数']):
        return df['行の文字数']
    
    #文字数に数値が入っている場合、そのまま代入
    elif str(df['文字数']).isdigit() == True: #簡単な例から場合分けする。 文字列か、数値か判定
        return int(df['文字数'])

    #ルール分けが難しい場合は直接数字で返す
    if '5語程度×2枠' in str(df['文字数']):#ここも語数だから直す
        return int(df['文字数'].replace('5語程度×2枠','10'))
    if '50字以上60字以内（日本語）' in str(df['文字数']):
        return int(df['文字数'].replace('50字以上60字以内（日本語）','50'))
    if '30字以上40字以内' in str(df['文字数']):
        return int(df['文字数'].replace('30字以上40字以内','30'))
    if '30～45語' in str(df['文字数']):#ここも語数だから直す
        return int(df['文字数'].replace('30～45語','30'))
    if '5～10語' in str(df['文字数']):#ここも語数だから直す
        return int(df['文字数'].replace('5～10語','5'))

    #数値に文字列を含む場合は文字列を除外し数値だけ返す。また意味不明は空白で返す
    for i in x:
        if i in str(df['文字数']):
            return make_sum(df['文字数'].replace(i,""))
    #「語」表記は5倍して返す。ただし、そのうち分野区分マスタに英文か和文かを示すダミー変数を作成して試したいので、ここは後で直す予定。
    for i in y:
        if i in str(df['文字数']):
            return int(df['文字数'].replace(i,""))
    else:
        return np.nan

    
df = raw_df

#定義したルールをapplyでdfの全てに対して一行ずつ適応している。applyはよく使う。
df['置換後の文字数']=df.apply(make_mojisu,axis = 1)

In [6]:
#関数を作る講義　文字数
#接頭語と接尾語 make_moji, make_sum
#1単語5文字
#正規表現調べる

def make_sum(input_str):
    if input_str =="":
        return np.nan
    else:
        input_list = input_str.split('＋')#+が半角か全角か異なる場合もあり。
        input_list_num = [int(s) for s in input_list] #リスト内包表記　復習
        return sum(input_list_num)
#5倍するラムダ関数
times_five = lambda x: x*5

#一行に対するルールを定義する 上から順に作成して、順次テストする
x = ['文字程度','文字（説明）','字程度（日本語）','文字','※解答枠６。まとめて10点', '単語？','太枠', 'ポ採\u30001枠', 'ポ採\u30003枠','\u3000','小枠2つ','2枠', '1枠', '3枠']
y = ["語程度","語"]
z = ['5語程度×2枠', '50字以上60字以内（日本語）','30字以上40字以内']
def make_mojisu(df):
    
    #行を含む場合は行数の文字列を代入
    if '行' in str(df['文字数']):
        return df['行の文字数']
    
    #文字数に数値が入っている場合、そのまま代入
    elif str(df['文字数']).isdigit() == True: #簡単な例から場合分けする。 文字列か、数値か判定
        return int(df['文字数'])

    #ルール分けが難しい場合は直接数字で返す
    if '5語程度×2枠' in str(df['文字数']):#ここも語数だから直す
        return int(df['文字数'].replace('5語程度×2枠','50'))
    if '50字以上60字以内（日本語）' in str(df['文字数']):
        return int(df['文字数'].replace('50字以上60字以内（日本語）','50'))
    if '30字以上40字以内' in str(df['文字数']):
        return int(df['文字数'].replace('30字以上40字以内','30'))
    if '30～45語' in str(df['文字数']):#ここも語数だから直す
        return int(df['文字数'].replace('30～45語','150'))
    if '5～10語' in str(df['文字数']):#ここも語数だから直す
        return int(df['文字数'].replace('5～10語','25'))

    #数値に文字列を含む場合は文字列を除外し数値だけ返す。また意味不明は空白で返す
    for i in x:
        if i in str(df['文字数']):
            return make_sum(df['文字数'].replace(i,""))
    #「語」表記は5倍して返す。ただし、そのうち分野区分マスタに英文か和文かを示すダミー変数を作成して試したいので、ここは後で直す予定。
    for i in y:
        if i in str(df['文字数']):
            return times_five(int(df['文字数'].replace(i,"")))
    else:
        return np.nan

    
# df = raw_df

#定義したルールをapplyでdfの全てに対して一行ずつ適応している。applyはよく使う。
df['置換後の文字数_1語5文字']=df.apply(make_mojisu,axis = 1)

In [7]:
"""ポイント数の修正"""
# df = raw_df
name_col = "ポイント数"
lst = []
df[name_col] = df[name_col].replace('\u3000',np.nan)
df[name_col] = df[name_col].replace('',np.nan)
df[name_col] = df[name_col].replace('2+7+8(ポ5)',"2+7+8")
df[name_col] = df[name_col].replace('2+4+6（ポ3）',"2+4+6")
                             
for i in df[name_col]:
    if type(i) == str:
        x = i.split('+')
        y = list(map(int, x))
        i = sum(y)
    lst.append(i)

df['置換後のポイント数'] = lst
# df_1 = df

In [8]:
"""配点の修正"""
# df = raw_df
name_col = "配点"
lst = []
df[name_col] = df[name_col].replace('\u3000',np.nan)
df[name_col] = df[name_col].replace('',np.nan)
df[name_col] = df[name_col].replace('5(2+2+1)+5',10)
df[name_col] = df[name_col].replace('2*2',4)
                             
for i in df[name_col]:
    if type(i) == str:
        if '+' in i:
            x = i.split('+')
        elif '＋' in i:
            x = i.split('＋')
        y = list(map(int, x))
        i = sum(y)
    lst.append(i)

df['置換後の配点'] = lst
# df_1 = df

In [9]:
df_1=df

### 自作したもの。メモ

In [84]:
"""
文字数を修正
もしも'行'を含んでいたら、行の文字数を代入
もしも'文字'を含んでいたら、文字の前の数字を代入
もしも'字'を含んでいたら、文字の前の数字を代入
もしも'語'を含んでいたら、文字の前の数字に5をかけて代入
例外は文字列として置換
"""

df = df_1
df['行の文字数'] = df['行の文字数'].fillna(0)
header = df.columns.values.tolist()
x= header.index('文字数')
y= header.index('行の文字数')

#例外除去　強引に置き換えているので、なんとかしたい。
#1語は5文字とした
df = df.replace({'文字数': {'※解答枠６。まとめて10点':np.nan,
                         'ポ採\u30001枠':np.nan, 
                         'ポ採\u30003枠':np.nan,
                         '\u3000':np.nan,
                         '単語？':np.nan,
                         '太枠':np.nan,
                         '2枠':np.nan,
                         '1枠':np.nan,
                         '3枠':np.nan,
                         '５語':25,
                         '小枠2つ':np.nan,
                         '10文字＋20文字':30,
                         '30～45語':150,
                         '5～10語':37.5,
                         '30字以上40字以内':35,
                         '5語程度×2枠':50}})

for i,n in enumerate(df['文字数']):
    if df.iat[i,y] != 0:
        df.iat[i, x] = df.iat[i, y]
    else:
        df.iat[i, x] = df.iat[i, x]
    if type(df.iat[i, x]) == str: #文字数列のi行数
        if  '行' in df.iat[i, x]:#行を含んでいたら、
            df.iat[i, x] = df.iat[i, y]#行の文字数列の値と置換
        elif  '文字' in df.iat[i, x]:
            df.iat[i, x] = n[0:n.index('文')]#150文字を150
        elif  '字' in df.iat[i, x]:
            df.iat[i, x] = n.replace("字","") 
        elif  '語' in df.iat[i, x]:
            df.iat[i, x] = int(n[0:n.index('語')]) *5

lst = [] #1+2+3などを6にする
for i in df['文字数']:
    if type(i) == str:
        x = i.split('+')
        y = list(map(int, x))
        i = sum(y)
    lst.append(i)
df['文字数'] = lst
    
# df_temp = df
df_1=df

ValueError: invalid literal for int() with base 10: '30程度（日本語）'

In [102]:
"""
ポイント数を修正
もしも配点が+を含んでいたら、ポイント数はその数+1
もしもポイント数がnanでなければ、そのまま
もしもポイント数がnanなら、1に置換
ヘッダーのインデックス
8 ポイント数
9 配点
"""

df = df_1

header = df.columns.values.tolist()
x= header.index('ポイント数')
y= header.index('配点')

for i,n in enumerate(df['配点']):
    if type(df.iat[i, y]) == str:
        if  '+' in df.iat[i, y]:
            df.iat[i, x] = n.count('+') + 1

for i,n in enumerate(df['ポイント数']):
    if type(df.iat[i, x]) == str:
        if  '+' in df.iat[i, x]:
            df.iat[i, x] = n.count('+') + 1

df['ポイント数'] = df['ポイント数'].replace(np.nan,1)
# df_temp=df
df_1=df

In [125]:
"""
配点を修正する
各文字列を取得
+でスプリット
各要素をsum
値を代入

"""
df = df_1
lst = []
df['配点'] = df['配点'].replace('\u3000',np.nan)
df['配点'] = df['配点'].replace('',np.nan)
df['配点'] = df['配点'].replace('5(2+2+1)+5',10)
                             
for i in df['配点']:
    if type(i) == str:
        x = i.split('+')
        y = list(map(int, x))
        i = sum(y)
    lst.append(i)

df['配点'] = lst
df_1 = df

ValueError: invalid literal for int() with base 10: '8＋12'

## 分野の修正

In [11]:
#分野マスターは分野の表記揺れを統一したもの。 CRLEA様から修正版が送られてきたら、csvにして再度使用
"""
data1から分野を抜き出したcsvファイルを読み込む
"""

name_csv = "bunyamaster_v3.1.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/rawdata"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_csv)#ファイルパスとファイル名

# csvファイルの読み込みと空のリストに追加
bunya = pd.read_csv(filepath_or_buffer = path_file, sep=",")
bunya = bunya.rename(columns={'分野名':'分野'})
bunya.head()

,科目コード,分野,分野名_修正v1,分野名_修正v2,ポイント採点,解答言語
0,10,評論,評論,評論,0,日本語
1,10,現代文・評論,評論,評論,0,日本語
2,10,現代文（評論）,評論,評論,0,日本語
3,10,現代文・評論（理由説明）,評論,評論,0,日本語
4,10,ポ評論,評論,評論,1,日本語


In [12]:
"""
2つのテーブルを結合させる
df_1:
bunya:分野の表記揺れをなおした列（分野名（修正））がある
"""
df_1b = pd.merge(df_1,bunya, on = ["分野","科目コード"],how = 'left')
# df_data2b.drop(['氏名', '登録試験点数'], axis=1)
df_1b.head()

,科目,大問番号,設問番号,割当1,分野,文字数,行の文字数,割当ごとのS枠数,ポイント数,配点,...,S枠\n(数字),科目コード,置換後の文字数,置換後の文字数_1語5文字,置換後のポイント数,置換後の配点,分野名_修正v1,分野名_修正v2,ポイント採点,解答言語
0,国語,1.0,3,01,ポ現代文・評論,60文字,NaN,1.0,NaN,6,...,NaN,10.0,60.0,60.0,NaN,6.0,評論,評論,1,日本語
1,国語,1.0,4,02,現代文・評論,40文字,NaN,1.0,NaN,5,...,NaN,10.0,40.0,40.0,NaN,5.0,評論,評論,0,日本語
2,国語,2.0,3,03,ポ現代文・小説,60文字,NaN,1.0,NaN,5,...,NaN,10.0,60.0,60.0,NaN,5.0,小説,小説,1,日本語
3,国語,2.0,4,04,現代文・小説,40文字,NaN,1.0,NaN,4,...,NaN,10.0,40.0,40.0,NaN,4.0,小説,小説,0,日本語
4,国語,3.0,3,05,古文,1行,12.0,1.0,NaN,5,...,NaN,10.0,12.0,12.0,NaN,5.0,古文,古文,0,NaN


## 指標の追加

### 欠損値の保管

In [131]:
"""
欠損値を分野別、科目別平均値で補完
1.　pandasでDFの形でデータを読み込む
2.　df.queryで分野2を条件に、カテゴリごとのDFに分ける
3.　分けたDFで、平均値で欠損値を埋める
4.　df.appendで分けたDFを結合
"""
df = df_1b.copy()
list_bunya = df['分野名_修正v2'].unique()
list_fillna = ['置換後の文字数','置換後のポイント数','置換後の配点']

df_ret=[]
for i in list_bunya:
    ret = df[df['分野名_修正v2'] == i]
    for j in list_fillna:
        ret[j] =ret[j].fillna(ret[j].mean())
    df_ret.append(ret)

dfs = df_ret[0] #集計結果を一つのDFに結合
for i in df_ret[1:]:
    dfs = dfs.append(i)

pandas_tool.summary(dfs)

/var/folders/ll/_gcfyf2137x3x6dwgb9nkj740000gp/T/ipykernel_3098/4154898545.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret[j] =ret[j].fillna(ret[j].mean())


(             企画ペース        最終ペース   S枠\n(数字)       科目コード     置換後の文字数  \
 count   345.000000   494.000000  37.000000  494.000000  491.000000   
 mean    286.594772   289.810662   1.756757   20.000000   28.504479   
 std     203.178502   295.762785   0.925125    8.031379   33.198252   
 min      45.000000    25.036229   1.000000   10.000000    1.000000   
 25%     150.000000   138.425000   1.000000   10.000000    2.750000   
 50%     250.000000   221.848810   1.000000   20.000000   17.000000   
 75%     360.000000   351.100000   3.000000   30.000000   44.500000   
 max    1800.000000  3266.300000   3.000000   30.000000  225.000000   
 
         置換後のポイント数      置換後の配点      ポイント採点           更新日  
 count  479.000000  494.000000  494.000000  4.940000e+02  
 mean     5.078153   14.853044    0.141700  2.022051e+07  
 std      4.010071   12.905487    0.349096  3.282359e+00  
 min      1.000000    2.000000    0.000000  2.022051e+07  
 25%      2.000000    5.000000    0.000000  2.022051e+07  
 50% 

In [65]:
dfs['置換後の文字数'].isna().any()

True

#### ランクの追加

In [8]:
#指標の追加
df_1 = df_1.eval('index4 = 最終ペース/企画ペース')
df_1 = df_1.eval('index5 = (企画ペース-最終ペース)/最終ペース')
df_1 = df_1.eval('index6 = 企画ペース/最終ペース')

In [9]:
df = df_1
df_colname = df_1.columns.tolist()
x = df_colname[10]
conditions = [
    (df[x] >= 600),
    (df[x] >= 500),
    (df[x] >= 400),
    (df[x] >= 300),
    (df[x] >= 200),
    (df[x] >= 100),
    (df[x] >= 0)
     ]

choices = ["600~", "500~600", "400~500", "300~400", "200~300", "100~200","0~100"]

df['企画ランク'] = np.select(conditions, choices, default = 0)

In [10]:
df = df_1
df_colname = df_1.columns.tolist()
x = df_colname[11]
conditions = [
    (df[x] >= 600),
    (df[x] >= 500),
    (df[x] >= 400),
    (df[x] >= 300),
    (df[x] >= 200),
    (df[x] >= 100),
    (df[x] >= 0)
     ]

choices = ["600~", "500~600", "400~500", "300~400", "200~300", "100~200","0~100"]

df['最終ランク'] = np.select(conditions, choices, default = 0)

In [108]:
df_1.head()

,科目,大問番号,設問番号,割当1,分野,文字数,行の文字数,割当ごとのS枠数,ポイント数,配点,企画ペース,最終ペース,採点回,学年,科目コード,index4,index5,index6,企画ランク,最終ランク
0,国語,1.0,3,01,ポ現代文・評論,60文字,NaN,1.0,NaN,6,150.0,143.4,冬,1,10.0,0.956000,0.046025,1.046025,100~200,100~200
1,国語,1.0,4,02,現代文・評論,40文字,NaN,1.0,NaN,5,450.0,449.8,冬,1,10.0,0.999556,0.000445,1.000445,400~500,400~500
2,国語,2.0,3,03,ポ現代文・小説,60文字,NaN,1.0,NaN,5,155.0,108.0,冬,1,10.0,0.696774,0.435185,1.435185,100~200,100~200
3,国語,2.0,4,04,現代文・小説,40文字,NaN,1.0,NaN,4,250.0,124.9,冬,1,10.0,0.499600,1.001601,2.001601,200~300,100~200
4,国語,3.0,3,05,古文,1行,12.0,1.0,NaN,5,600.0,846.9,冬,1,10.0,1.411500,-0.291534,0.708466,600~,600~


In [10]:
import collections

c = collections.Counter(df_1['企画ランク'])
print(c)

Counter({'100~200': 47, '200~300': 46, '300~400': 40, '400~500': 20, '0~100': 12, '500~600': 9, '600~': 8})


In [82]:
df_1.dtypes

科目           object
大問番号         object
設問番号         object
割当1          object
分野           object
文字数          object
行の文字数        object
割当ごとのS枠数    float64
ポイント数        object
配点           object
企画ペース       float64
最終ペース       float64
採点回          object
学年           object
科目コード       float64
dtype: object

In [92]:
"""
Excelの書き出し data1
"""

name_excel_output = "①設問情報（分野、文字数等の情報、企画ペース）_0522更新_edited.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_1.to_excel(writer, sheet_name='data1')

## データの書き出し

In [132]:
"""
Excelの書き出し fillna 欠損値補完
"""

df = dfs

name_excel_output = "crlea1_dm_0526_fillna.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

df.to_csv('{}/{}'.format(path_folder,name_excel_output),encoding='utf-8-sig')

name_excel_output = "crlea1_dm_0526_flllna.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#Excelが置いてあるフォルダパス


with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig')

In [13]:
"""
Excelの書き出し data1b　欠損値そのまま
"""

df = df_1b

name_excel_output = "crlea_data1_0601.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig')

In [152]:
"""
csvの書き出し data1b
"""

df = df_1b

# df= df[df['科目',
#           '採点回',
#           '学年',
#           '科目コード',
#           '分野名（修正）',
#           '割当1',
#           '文字数',
#           'ポイント数',
#           '配点',
#           '企画ペース',
#           '最終ペース',
#           'index4',
#           'index5',
#           'index6',
#           '企画ランク',
#           '最終ランク',
#           '割当ごとのS枠数',
#           'ポイント採点']]

name_excel_output = "crlea1_dm_0525_fillna.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

df.to_csv('{}/{}'.format(path_folder,name_excel_output),encoding='utf-8-sig')

## 割当を編集する

In [59]:
df_1['割当1'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       'K1', 'K2', 'Kx', 'K3', 'K4', 'Kt', 'K5', '04-4', '04-5', '04-6',
       '04-7', '04-8', 'Ks', '03B3', '04B4', '04B5', '04B6', '04B7',
       '04B8', '05A3', '06A4', '06A5', '06A6', '06A7', 'Ku', 'Kv', 'Kw',
       'Kp', 'K6', '01 ', '02 ', '03 ', '04 ', '05 ', '06 ', 'K7', 'K8',
       'K9', 'nan', '03-4', '03-5', '03-6', '1', '2', '3', '44', '45',
       '46', '47', '48', '5', '64', '66', '371', '12', '01-1', '01-2',
       '01-3', '02-4', '02-5', '08-3', '08-4', '13-3', '13-4', '13-5',
       '13-6', '13-7', '13-8', '13-9', '13-10', '13', 'Ka', '79'],
      dtype=object)

In [136]:
df_1.columns.tolist()

['科目',
 '大問番号',
 '設問番号',
 '割当',
 '分野',
 '文字数',
 'ポイント数',
 '配点',
 '割当ごとのS枠数',
 '企画ペース',
 '最終ペース',
 '採点回',
 '学年',
 '科目コード']

In [249]:
#data1の割当をエクセルで手作業で修正したものを読み込む
"""
ファイル内のすべてのシート名をループして、
各シートをデータフレームとしてdfsというリストの各要素に保存。
dfs[X]で任意のデータフレームとして機能。
"""
name_excel = "data1_edited_manual_v2.1.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_excel)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
df = pd.read_excel(path_file, sn_list[0], index_col=0)
df = df.rename(columns={'割当1':'割当'})
#学年のデータ型をintからobjectに
df["学年"] = df["学年"].astype('object')
df= df[~df["科目コード"].isna()]
# df = df[["科目コード","採点回","学年",'割当','企画ペース', '最終ペース']]
# df['joint_key'] = df['科目コード'].astype(object) + 　df['採点回']+df['学年'].astype(object)+ df['割当']
df_1m =df

In [251]:
"""
Excelの書き出し df_1m
"""

name_excel_output = "df_1m.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_1m.to_excel(writer, sheet_name='temp')

# データ2の前処理

In [29]:
#data2は②をエクセルでコピペして切り出したもの。分野ごとの企画ペースと最終ペースが記載してある。
"""
ファイル内のすべてのシート名をループして、
各シートをデータフレームとしてdfsというリストの各要素に保存。
dfs[X]で任意のデータフレームとして機能。
"""
# import pandas as pd
name_excel = "data2.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_excel)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list_data2 = input_file.sheet_names
sn_list_data2 = sn_list_data2[0:len(sn_list_data2)-4]
#配列ループ
df = [] #空のリスト
for i in range(0,3): #夏、秋、冬

    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list_data2[i], index_col=None)
    #ret
    ret = ret.rename(columns={'採点回':'学年','割当単位':'割当'})
    ret["採点回"]= sn_list_data2[i][0]#sn_listのiつ目の要素から1番目の文字
    # ret["学年"]=int(sn_list_data1[i][1])#sn_listのiつ目の要素から2番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)

    
"""
集計したデータを縦に結合する。
df = df1.append(df2)
"""
dfs = df[0] #集計結果を一つのDFに結合
for i in df[1:]:
    dfs = dfs.append(i)

"""
作成したデータフレームの編集
・列名変更
・変数名変更
・行の削除
"""
#列名確認
# df.columns.values

dfs = dfs.replace({'学年': {'1年生':1,'2年生':2,'3年生':3,'客専':np.nan},
                   '割当': {'-':np.nan,'KS':np.nan,'パート分けナシ':np.nan,'?':np.nan},
                   '分野': {'-':np.nan,'KS':np.nan,'パート分けナシ':np.nan,'?':np.nan}})
dfs= dfs[~dfs["学年"].isna()]
dfs= dfs[~dfs["割当"].isna()]
dfs= dfs[~dfs["分野"].isna()]
dfs = dfs.rename(columns={'3始まり\nスタッフコード':'採点者コード'})
dfs = dfs[['スタッフコード','採点者コード', '科目コード', '採点回', '学年', '割当','分野']]
dfs = dfs.rename(columns={'割当':'割当2'})
dfs['割当2'] = dfs.loc[:, "割当2"].astype('object')
df_data2 =dfs
df_data2.head()

,スタッフコード,採点者コード,科目コード,採点回,学年,割当2,分野
0,1051286194,3051286194,10,夏,1.0,01,評論
1,1051284876,3051284876,10,夏,1.0,01,評論
3,1060309944,3060309944,10,夏,1.0,01,評論
4,1060311559,3060311559,10,夏,1.0,01,評論
5,1060320722,3060320722,10,夏,1.0,01,評論


In [30]:
"""
Excelの書き出し data2
"""

name_excel_output = "data2_edited.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_data2.to_excel(writer, sheet_name='data2')

In [222]:
"""
Excelの書き出し data2b
"""

name_excel_output = "data2b_edited.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_data2b.to_excel(writer, sheet_name='data2b')

## データ3の前処理

In [2]:
#data3は③をリネームしたエクセル。採点コードごとのペースが記載してある。
"""
ファイル内のすべてのシート名をループして、各シートをデータフレームとしてdfというリストの各要素に保存。df[X]で任意のデータフレームとして機能。
"""
import pandas as pd
name_excel = "data3.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_excel)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list= input_file.sheet_names
sn_list = sn_list[0:len(sn_list)-2]
#配列ループ
df = [] #空のリスト
for i in range(0,len(sn_list)): 

    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i], index_col=None)
    #ret
    ret["採点回"]= sn_list[i][0]#sn_listのiつ目の要素から1番目の文字
    ret["学年"]=int(sn_list[i][1])#sn_listのiつ目の要素から2番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)

    
"""
集計したデータを縦に結合する。
df = df1.append(df2)
"""
dfs = df[0] #集計結果を一つのDFに結合
for i in df[1:]:
    dfs = dfs.append(i)

# map_dictionary ={"国語" : 10, "数学" :20, "英語" : 30} 
# dfs['科目コード'] = dfs['科目'].map(map_dictionary) 
dfs = dfs[['科目コード','採点者コード', '採点回', '学年', '割当','ペース']]
dfs = dfs.rename(columns={'割当':'割当3'})
dfs=dfs[dfs['科目コード'].isin([10, 20,30])]#科目コード10、20、30を抽出
df_data3 = dfs
df_data3.head()

,科目コード,採点者コード,採点回,学年,割当3,ペース
0,10.0,3.050016e+09,夏,1,1.0,209.083637
1,10.0,3.050727e+09,夏,1,1.0,221.578735
2,30.0,3.050742e+09,夏,1,5.0,101.567398
3,10.0,3.051285e+09,夏,1,1.0,173.823108
4,10.0,3.051286e+09,夏,1,1.0,1649.259462


In [92]:
"""
Excelの書き出し data3
"""

name_excel_output = "data3_edited.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_data3.to_excel(writer, sheet_name='data3')

## データ2とデータ3の結合

In [158]:
"""
df_data2b:採点者コード、科目コード、採点回、学年、割当2、分野を記載したテーブル
df_data3:採点者コード、科目コード、採点回、学年、割当3、ペース
2つのテーブルを結合させる
科目コード、採点回、学年、割当をつなげた列を作成する
"""
df1 = df_data2b
df2 = df_data3
df = pandas_tool.merge(df1,df2,key=["採点者コード","科目コード","採点回","学年"],how = 'right',view=True)
# df = df.rename(columns={'割当2':'割当'})
df['割当3a']=df['割当3'].astype(str)
map_dictionary ={"1.0" : "01",
                  "2.0" :"02",
                  "3.0" : "03",
                 "4.0" : "04",
                  "5.0" :"05",
                  "6.0" : "06",
                 "7.0" : "07",
                  "8.0" :"08",
                  "9.0" : "09",
                 "10.0" : "10",
                  "11.0" :"11",
                  "12.0" : "12",
                  "13.0" : "13"} 
df['割当3a'] = df['割当3a'].map(map_dictionary) 
df['割当2'] = df['割当2'].mask(df['割当2'] == 30, 20)
df['割当'] = df['割当2'].mask(df['割当2'] == 20, df['割当3a'])

#学年のデータ型をintからobjectに
df["学年"] = df["学年"].astype('object')
# map_dictionary ={"1.0" : "1", "2.0" :"2", "3.0" : "3"} 
# df['学年'] = df['学年'].map(map_dictionary) 
df = df[["スタッフコード","採点者コード","科目コード","採点回","学年",'割当','割当2','割当3','割当3a','ペース', '分野', '分野_v2', 'ポイント採点', '備考']]
df_2b3 = df

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:15739
df2のオブザベーション:12376
mergeのオブザベーション:12376


In [89]:
"""
Excelの書き出し df_2b3
"""

name_excel_output = "df_2b3.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_2b3.to_excel(writer, sheet_name='temp')

## データ23とデータ1の結合

In [252]:
"""
df_2b3:採点者コード、科目コード、採点回、学年、割当、分野、ペース
df_1m:科目コード、採点回、学年、割当、企画ペース、最終ペース
"""
df1 = pd.read_excel(r"/Users/s.ogura/Documents/CRLEA/data/20220428/df_2b3.xlsx",
                    dtype = {'割当': object}, index_col=0)
df2 = pd.read_excel(r"/Users/s.ogura/Documents/CRLEA/data/20220428/df_1m.xlsx", index_col=0)

#科目コード、採点回、学年、割当をつなげた列を作成する

df1["temp1"] = df1['採点回'].astype(str).str.cat(df1['学年'].astype(str))
df1["temp2"] = df1['科目コード'].astype(str).str.cat(df1['割当'].astype(str))
df1["temp3"] = df1['temp1'].str.cat(df1['temp2'])

df2["temp1"] = df2['採点回'].astype(str).str.cat(df2['学年'].astype(str))
df2["temp2"] = df2['科目コード'].astype(str).str.cat(df2['割当'].astype(str))
df2["temp3"] = df2['temp1'].str.cat(df2['temp2'])

#重複する列を削除
df1 = df1[["temp3","スタッフコード","採点者コード","採点回","学年","科目コード",'割当',"ペース","分野","分野_v2","ポイント採点","備考"]]
df2 = df2[["temp3",'文字数','ポイント数','配点',"企画ペース","最終ペース"]]

#2つのテーブルを結合させる
df = pandas_tool.merge(df1,df2,key=["temp3"],how = 'left',view=True)

df_1m2b3 = df

df1のキー重複
12126
df2のキー重複
1
df1のオブザベーション:12376
df2のオブザベーション:351
mergeのオブザベーション:12376


In [253]:
"""
Excelの書き出し df_1m2b3
"""

name_excel_output = "df_1m2b3.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_1m2b3.to_excel(writer, sheet_name='temp')

In [147]:
"""
一応保管
df_2b3:採点者コード、科目コード、採点回、学年、割当、分野、ペース
df_1m:科目コード、採点回、学年、割当、企画ペース、最終ペース
複数キーで結合。値のいくつかのデータタイプが異なるため、上手く結合できていない。
"""
df1 = pd.read_excel(r"/Users/s.ogura/Documents/CRLEA/data/20220428/df_2b3.xlsx",
                    dtype = {'割当': object}, index_col=0)

df2 = pd.read_excel(r"/Users/s.ogura/Documents/CRLEA/data/20220428/df_1m.xlsx", index_col=0)
df2['割当1']=df2['割当']

#2つのテーブルを結合させる
df = pandas_tool.merge(df1,df2,key=["採点回","学年","科目コード","割当"],how = 'left',view=True)

#重複する列を削除
# df1 = df1[["temp3","スタッフコード","採点者コード","ペース","分野","分野_v2","ポイント採点","備考"]]
# df2 = df2[["temp3","企画ペース","最終ペース"]]

df_1m2b3 = df

df1のキー重複
12122
df2のキー重複
1
df1のオブザベーション:12376
df2のオブザベーション:351
mergeのオブザベーション:12376


In [148]:
"""
Excelの書き出し df_1m2b3　複数キー
"""

name_excel_output = "df_1m2b3_key.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_1m2b3.to_excel(writer, sheet_name='temp')

##　データ123とデータ4の結合

In [254]:
#dfの読み込み
df1 = df_1m2b3
df2 = pd.read_excel(r"/Users/s.ogura/Documents/CRLEA/data/received_20220412/④採用者情報_個人に関わる詳細.xlsx",
                    dtype = {'スタッフコード': object}, index_col= None)
#2つのテーブルを結合させる
df = pandas_tool.merge(df1,df2,key=["スタッフコード"],how = 'left',view=True)
df_1m2b34 = df

df1のキー重複
5722
df2のキー重複
0
df1のオブザベーション:12376
df2のオブザベーション:14772
mergeのオブザベーション:12376


In [255]:
"""
Excelの書き出し df_1m2b34
"""

name_excel_output = "df_1m2b34.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_1m2b34.to_excel(writer, sheet_name='temp')

In [258]:
df_1m2b34.to_csv(r'/Users/s.ogura/Documents/CRLEA/data/20220428/CRLEA_datamart_0510.csv',
                 header=True, index=False, encoding='cp932')